# Lightweight Fine-Tuning to a Foundation Model

## In this project, we will describe choices for each of the following
- Load a pre-trained model and evaluate its performance
- PEFT technique: LoRA
- Fine-tuning dataset: Hotel Bookings  Hotel Bookings.csv

In [4]:
pip install transformers datasets evaluate peft accelerate

Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType, AutoPeftModelForSequenceClassification
import evaluate

In [8]:
dataset = load_dataset("ag_news")
dataset = dataset.map(lambda x: {"text": x["text"][:512]})  # truncate long text

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

C:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Priyanshu Prakash\.cache\huggingface\hub\datasets--ag_news. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [9]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Priyanshu Prakash\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    do_eval=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

print("✅ Base Model Evaluation:")
trainer.evaluate()

✅ Base Model Evaluation:


{'eval_loss': 1.3946335315704346,
 'eval_model_preparation_time': 0.0022,
 'eval_accuracy': 0.2181578947368421,
 'eval_runtime': 964.4962,
 'eval_samples_per_second': 7.88,
 'eval_steps_per_second': 0.492}

In [18]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_lin", "v_lin"]  # or ["query", "value"] depending on model
)

In [22]:
pip install -U transformers

Note: you may need to restart the kernel to use updated packages.


In [30]:
import transformers
print(transformers.__version__)

4.51.0


In [32]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

In [40]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=tokenized_dataset["test"].select(range(500)),
    compute_metrics=compute_metrics,
)

In [44]:
trainer.train()

Step,Training Loss
10,1.352100
20,1.190800
30,0.916800


KeyboardInterrupt: 

In [52]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_dataset["test"].select(range(500)),
    compute_metrics=compute_metrics,
)

print("✅ PEFT Model Evaluation:")
trainer.evaluate()


✅ PEFT Model Evaluation:


{'eval_loss': 0.7567319273948669,
 'eval_model_preparation_time': 0.0158,
 'eval_accuracy': 0.852,
 'eval_runtime': 146.0628,
 'eval_samples_per_second': 3.423,
 'eval_steps_per_second': 0.11}

### Interference

In [55]:
text = "Apple is releasing a new iPhone this year."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
outputs = model(**inputs)
predicted_label = torch.argmax(outputs.logits, dim=-1).item()
print("Predicted label:", predicted_label)

Predicted label: 3
